## Initialize

In [31]:

import requests
import pandas as pd
import os
import sys
from datetime import datetime
from ipywidgets import Dropdown
import json
import base64
import re
import requests

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
import src.esh_client as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path
base_url = get_base_url()
root_path = get_root_path()
example_name = sys.path[0].split(os.sep)[-2]
headers = {'Content-Type': 'application/json'}
url =

def execute_search_query(query):
    r = requests.post(
        f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[
                ['title', 'text']]
            df.rename(columns={'title': 'Title',
                      'text': 'Text'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())

## Choose file folder

In [32]:
folder_path = os.path.join('..')
options = next(os.walk(folder_path))[1]
selected = Dropdown(
    options=options,
    description='Select:',
    value = None,
    disabled=False,
)
selected

Dropdown(description='Select:', options=('externDocuments', 'internDocuments', 'notebooks'), value=None)

## Take all files from folder

In [33]:
if selected.value:
    payload_list = []
    #Get files path
    current_path = os.path.join(sys.path[0], '..',selected.value)
    #Iterate about the folder with all files
    for filename in os.listdir(current_path):
        #Read PDF File and encode to base64
        with open(os.path.join(current_path, filename), 'rb') as f: # open in readonly mode
            filename = filename.replace('.pdf','')
            print(filename)
            #encode to base64
            encoded_string = base64.b64encode(f.read())
            encoded_doc = encoded_string.decode('ascii')
            #Append to JSON
            payload = json.dumps({
                'Document': [
                    {
                        'title': filename,
                        'text': encoded_doc
                    }
                ]
                        }, indent=2
            )
            payload_list.append(payload)
            
else:
    print('No folder selected for PDF files')    

audi_report
bmw_report
mercedes_report
porsche_report
volkswagen_report


## Post data to server

In [ ]:
response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)